In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2

In [ ]:
dataframe = pd.read_csv('../input/facial-expression/fer2013.csv') 

In [ ]:
dataframe.head()

In [ ]:
len(dataframe.iloc[0]['pixels'].split()) # 48 * 48

In [ ]:
target_emotions = ['Anger', 'Neutral', 'Fear', 'Happy', 'Sad', 'Surprise']

In [ ]:
image = dataframe.iloc[0]['pixels'].split()

In [ ]:
image = [int(i) for i in image]

In [ ]:
image = np.array(image)

In [ ]:
image = image.reshape(48,48)

In [ ]:
image.shape

In [ ]:
X = []
y = []

In [ ]:
def getData(path):
    anger = 0
    fear = 0
    sad = 0
    happy = 0
    surprise = 0
    neutral = 0
    dataframe = pd.read_csv(path)
    
    X = []
    y = []    
    
    for i in range(len(dataframe)):
        if dataframe.iloc[i]['emotion'] != 1:
            if dataframe.iloc[i]['emotion'] == 0:
                if anger <= 4000:            
                    y.append(dataframe.iloc[i]['emotion'])
                    im = dataframe.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    anger += 1
                else:
                    pass
                
            if dataframe.iloc[i]['emotion'] == 2:
                if fear <= 4000:            
                    y.append(dataframe.iloc[i]['emotion'])
                    im = dataframe.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    fear += 1
                else:
                    pass
                
            if dataframe.iloc[i]['emotion'] == 3:
                if happy <= 4000:            
                    y.append(dataframe.iloc[i]['emotion'])
                    im = dataframe.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    happy += 1
                else:
                    pass
                
            if dataframe.iloc[i]['emotion'] == 4:
                if sad <= 4000:            
                    y.append(dataframe.iloc[i]['emotion'])
                    im = dataframe.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    sad += 1
                else:
                    pass
                
            if dataframe.iloc[i]['emotion'] == 5:
                if surprise <= 4000:            
                    y.append(dataframe.iloc[i]['emotion'])
                    im = dataframe.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    surprise += 1
                else:
                    pass
                
            if dataframe.iloc[i]['emotion'] == 6:
                if neutral <= 4000:            
                    y.append(dataframe.iloc[i]['emotion'])
                    im = dataframe.iloc[i]['pixels']
                    im = [int(x) for x in im.split()]
                    X.append(im)
                    neutral += 1
                else:
                    pass

            
            
    return X, y  
    

In [ ]:
X, y = getData('../input/facial-expression/fer2013.csv')

In [ ]:
np.unique(y, return_counts=True)

In [ ]:
X = np.array(X)/255.0
y = np.array(y)

In [ ]:
X.shape, y.shape

In [ ]:
y_output = []
for i in y:
    if i != 6:
        y_output.append(i)
        
    else:
        y_output.append(1)

In [ ]:
np.unique(y_output, return_counts=True)

In [ ]:
for i in range(5):
    r = np.random.randint((1), 24000, 1)[0]
    plt.figure()
    plt.imshow(X[r].reshape(48,48), cmap='gray')
    plt.xlabel(target_emotions[y_output[r]])

In [ ]:
X = X.reshape(len(X), 48, 48, 1)

In [ ]:
X.shape

In [ ]:
from tensorflow.keras.utils import to_categorical
y_new = to_categorical(y_output, num_classes=6)

In [ ]:
len(y_output), y_new.shape

In [ ]:
y_output[150], y_new[150]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from tensorflow.keras.layers import BatchNormalization


In [ ]:
model = Sequential()


input_shape = (48,48,1)


model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
model.add(Conv2D(64, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
model.add(Conv2D(128, (5, 5),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

## (15, 15) --->  30
model.add(Flatten())
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X, y_new, epochs=22, batch_size=64, shuffle=True, validation_split=0.2)

In [ ]:
model.summary()

In [ ]:
model.save('model.h5')